# Implementacion subsistemas de notificaciones

In [1]:
import uuid
from datetime import datetime, timedelta, timezone

import redis
import pandas as pd

bd = redis.Redis(
    host="localhost",
    port=6379,
    db=0,
    decode_responses=True,
)

## Helpers

### Primero definiremos unos helpers que nos permitiran respetar el modelo planteado en la documentacio

In [2]:
def notif_hash_key(notifId: str) -> str:
    return f"notif:{notifId}"

def user_scheduled_key(userId: str, ts: int) -> str:
    return f"notif:{userId}:scheduled:{ts}"

def user_queue_key(userId: str) -> str:
    return f"notif:{userId}:queue"

def user_delivered_key(userId: str, notifId: str) -> str:
    return f"notif:{userId}:delivered:{notifId}"

def user_prefs_key(userId: str) -> str:
    return f"notif:preferences:{userId}"

## Funciones de escritura

1. Crear la funcion para crear la notificacion. Para ello creamos un modelo sencillo de:

- UuId
- Hacia quien esta definida la notificacion (UserId) para marcar la relacion con usuario
- El tipo de la notificacion, para simplificar aqui se utilizo un string que puede ser de cualquier tipo, pero en un futuro se puede definir tipos especificos
- Payload que contiene el mensaje de la notificacion
- Fechas de cuando se mando y cuando se creo
- Estado para marcar recibida o si esta pending de mandar

In [3]:
def create_notification( userId: str, notifType: str, payload: dict, sendAt: datetime, ttlSeconds: int, status: str = "pending",):
    notifId = str(uuid.uuid4())
    notifKey = notif_hash_key(notifId)

    # Hash de la notificación
    notificacion = {
        "notifId": notifId,
        "userId": userId,
        "type": notifType,
        "payload": str(payload),
        "createdAt": datetime.now(timezone.utc).isoformat(),
        "sendAt": sendAt.isoformat(),
        "status": status,
    }

    # Guardar notificación
    bd.hset(notifKey, mapping = notificacion)

    # TTL sobre la notificación
    bd.expire(notifKey, ttlSeconds)

    # Registrar como "programada" por usuario
    time = int(sendAt.timestamp())
    schedule = user_scheduled_key(userId, time)
    bd.set(schedule, notifId)
    bd.expire(schedule, ttlSeconds)

    # Encolar en la queue del usuario
    bd.rpush(user_queue_key(userId), notifId)

    return notifId

2. Funcion para actualizar la notificacion de que fue recibida

In [4]:
def mark_notification_delivered(userId: str, notifId: str, seen: bool = False):
    notifKey = notif_hash_key(notifId)

    status = "sent"
    if seen:
        status = "seen"

    bd.hset(notifKey, "status", status)
    bd.hset(notifKey, "deliveredAt", datetime.now(timezone.utc).isoformat())

    delivered = user_delivered_key(userId, notifId)
    bd.set(delivered, status)

    bd.expire(delivered, 7 * 24 * 3600)

3. Funcion para marcar las preferencias del usaurio

In [5]:

def set_user_notification_preferences(userId: str, **preferences):
    preference = user_prefs_key(userId)
    mapping = {k: str(v) for k, v in preferences.items()}
    bd.hset(preference, mapping=mapping)

## Funciones de Lectura

1. Muestra la lista de notificaciones en la cola del usuario

In [6]:
def get_user_queue(userId: str):
    return bd.lrange(user_queue_key(userId), 0, -1)


2. Muestra los detalle de una lista de notificaciones

In [7]:
def get_notifications_details(notifIds):
    notificaciones = []
    for nid in notifIds:
        data = bd.hgetall(notif_hash_key(nid))
        if data:
            notificaciones.append(data)
    if not notificaciones:
        return pd.DataFrame()
    return pd.DataFrame(notificaciones)

3. Devuelve una lista con ordenada con las notificaciones pendiantes del usuario

In [8]:
def get_user_scheduled(user_id: str):
    pattern = f"notif:{user_id}:scheduled:*"
    scheduele = []
    for key in bd.scan_iter(pattern):
        parts = key.split(":")
        ts = int(parts[-1])
        notif_id = bd.get(key)
        scheduele.append(
            {
                "key": key,
                "timestamp": ts,
                "sendAt_dt": datetime.fromtimestamp(ts, timezone.utc),
                "notifId": notif_id,
            }
        )

    if not scheduele:
        return pd.DataFrame()

    scheduele = sorted(scheduele, key=lambda x: x["timestamp"])
    return pd.DataFrame(scheduele)

## Pruebas

Primero podemos hacer una limpieza de la tabla para mostrar con mas precision el funcionamiento del codigo

In [9]:

for key in bd.scan_iter("notif:*"):
    bd.delete(key)
print("Claves borradas exitosamente")

ConnectionError: Error 10061 connecting to localhost:6379. No connection could be made because the target machine actively refused it.

Aqui hicimos un pequeño script para insertar datos en la tabla, las notificaciones para un usuario

In [ ]:
# Crear algunas notificaciones
now = datetime.now(timezone.utc)

notificaciones = []
notificaciones.append(
    create_notification(
        userId="user_1",
        notifType="streakReminder",
        payload={"message": "¡No pierdas tu racha de 10 días!", "lang": "es"},
        sendAt=now + timedelta(minutes=5),
        ttlSeconds=3600,
    )
)
notificaciones.append(
    create_notification(
        userId="user_1",
        notifType="lessonPending",
        payload={"message": "Te espera una lección de inglés", "lang": "en"},
        sendAt=now + timedelta(minutes=30),
        ttlSeconds=7200,
    )
)
notificaciones.append(
    create_notification(
        userId="user_2",
        notifType="achievement",
        payload={"message": "¡Nuevo logro: Unidad 3 completada!", "lang": "es"},
        sendAt=now + timedelta(minutes=1),
        ttlSeconds=3600,
    )
)

print("Notificaciones creadas exitosamente")


y luego aqui insertamos las preferencias

In [ ]:

# Preferencias de usuario
set_user_notification_preferences(
    "user_1",
    emailEnabled=True,
    pushEnabled=True,
    quietHours="23-07",
)
set_user_notification_preferences(
    "user_2",
    emailEnabled=False,
    pushEnabled=True,
    quietHours="00-00",
)

print("Preferencias creadas exitosamente")


In [ ]:
def get_user_preferences(userId: str):
    return bd.hgetall(user_prefs_key(userId))

print("Preferencias user_1:", get_user_preferences("user_1"))
print("\nPreferencias user_2:", get_user_preferences("user_2"))

Por utlimo aqui tenemos unas pruebas para mostrar como nos muestra las notificaciones del usuario y luego mostramos los detalles de cada una

In [ ]:
user = "user_1"

cola = get_user_queue(user)
print("Queue de usuario:", user, ", ", cola)

notificaciones = get_notifications_details(cola)
print ("\nLas notificaciones son:")
notificaciones

Aqui es como una version simplificada que nos muestra en menor detalle las notififcaciones del usuario con el TTL

In [ ]:
get_user_scheduled("user_1")


Y por ultimo aqui mostramos como entregamos la notificacion y mostramos como se muestra el estado en entregado

In [ ]:
# Marcamos como entregada y vista la primera notificación de user_1
if cola:
    first_notif = cola[0]
    mark_notification_delivered("user_1", first_notif, seen=True)

    delivered_key = user_delivered_key("user_1", first_notif)
    print("Clave delivered:", delivered_key)

    bd.hgetall(notif_hash_key(first_notif))


In [ ]:
user = "user_1"

cola = get_user_queue(user)
print("Queue de usuario:", user, ", ", cola)

notificaciones = get_notifications_details(cola)
print ("\nLas notificaciones son:")
notificaciones